In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt

In [4]:
spark = SparkSession.builder.appName('Ex1').getOrCreate()

In [6]:
data = spark.read.csv('./smsspamcollection/SMSSpamCollection',sep='\t',inferSchema=True)

In [7]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [8]:
from pyspark.sql.functions import col, count, when, isnan, isnull

In [9]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [10]:
data.show(3)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
+-----+--------------------+
only showing top 3 rows



## Clean Data

In [11]:
from pyspark.sql.functions import length

In [12]:
data = data.withColumn('length',length(data['text']))

In [13]:
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [14]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



### Feature Transformation

In [17]:
from pyspark.ml.feature import RegexTokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [18]:
tokenizer = RegexTokenizer(inputCol="text", outputCol="token_text", pattern='[^A-Za-z0-9]+',gaps=True)
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [19]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [20]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### Pipeline

In [22]:
from pyspark.ml import Pipeline

In [35]:
data_prep_pipe = Pipeline(stages=[tokenizer,stopremove,count_vec,idf,clean_up,ham_spam_to_num])
#data_prep_pipe = Pipeline(stages=[tokenizer,stopremove,count_vec,idf])
cleaner = data_prep_pipe.fit(data)
clean_data = cleaner.transform(data)

### Train model and Evaluate

In [36]:
from pyspark.ml.classification import NaiveBayes

In [37]:
(training,testing) = clean_data.randomSplit([0.7,0.3], seed=142)

In [41]:
nb = NaiveBayes(labelCol='label')

In [42]:
predictor = nb.fit(training)

In [43]:
test_results = predictor.transform(testing)

In [45]:
test_results.show()

+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|class|                text|length|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|label|       rawPrediction|         probability|prediction|
+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|  ham| &lt;#&gt;  mins ...|    51|[lt, gt, mins, bu...|[lt, gt, mins, st...|(8620,[7,8,33,127...|(8620,[7,8,33,127...|(8621,[7,8,33,127...|  0.0|[-232.70971977310...|[1.0,1.6424908202...|       0.0|
|  ham| and  picking the...|    41|[and, picking, th...|[picking, various...|(8620,[498,799,22...|(8620,[498,799,22...|(8621,[498,799,22...|  0.0|[-211.40766265815...|[0.99999928927889...|       0.0|


In [46]:
# Create a confusion matrix
test_results.groupBy('class', 'prediction').count().show()

+-----+----------+-----+
|class|prediction|count|
+-----+----------+-----+
|  ham|       1.0|   53|
| spam|       1.0|  209|
| spam|       0.0|    9|
|  ham|       0.0| 1379|
+-----+----------+-----+



In [47]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [49]:
acc_eval = MulticlassClassificationEvaluator(labelCol='label')
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting: {}".format(acc))

Accuracy of model at predicting: 0.9638533204384269


### Logistic Regression

In [50]:
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression

In [51]:
lg = LogisticRegression(maxIter=10, regParam=0.1, elasticNetParam=0,labelCol='label')

In [52]:
predictor_lg = lg.fit(training)

In [53]:
test_results_lg = predictor_lg.transform(testing)

In [54]:
# Create a confusion matrix
test_results_lg.groupBy('class', 'prediction').count().show()

+-----+----------+-----+
|class|prediction|count|
+-----+----------+-----+
| spam|       1.0|  165|
| spam|       0.0|   53|
|  ham|       0.0| 1432|
+-----+----------+-----+



In [55]:
acc_eval = MulticlassClassificationEvaluator(labelCol='label')
acc = acc_eval.evaluate(test_results_lg)
print("Accuracy of model at predicting: {}".format(acc))

Accuracy of model at predicting: 0.9659481127286582
